In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("..")
project_path = os.path.dirname(os.getcwd())
png = pd.read_csv(
    project_path + "/data/prices/outputs/2009/RTFP_country_2009_2022-08-17.csv")
png["date"] = pd.to_datetime(png["date"], dayfirst=True)

In [3]:
png

,Open,High,Low,Close,Inflation,country,ISO3,date
0,0.690540,0.742082,0.649058,0.695159,NaN,Papua New Guinea,PNG,2009-01-01
1,0.700222,0.747046,0.653399,0.709033,NaN,Papua New Guinea,PNG,2009-02-01
2,0.712256,0.764049,0.660462,0.716254,NaN,Papua New Guinea,PNG,2009-03-01
3,0.718400,0.773517,0.663282,0.716188,NaN,Papua New Guinea,PNG,2009-04-01
4,0.719868,0.776872,0.662863,0.722852,NaN,Papua New Guinea,PNG,2009-05-01
...,...,...,...,...,...,...,...,...
159,1.298422,1.385601,1.211244,1.255384,1.588927,Papua New Guinea,PNG,2022-04-01
160,1.274761,1.357783,1.191738,1.298791,8.031285,Papua New Guinea,PNG,2022-05-01
161,1.311183,1.397665,1.224701,1.296858,7.356945,Papua New Guinea,PNG,2022-06-01
162,1.308152,1.395347,1.220957,1.357078,12.759345,Papua New Guinea,PNG,2022-07-01


## Annual Food Price
### CandleStick - Bokeh

In [5]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Slider, CustomJS, \
                HoverTool, CrosshairTool, CDSView, BooleanFilter, BoxZoomTool, \
                PanTool, ResetTool
from bokeh.layouts import column

output_file("candlestick-bokeh.html", title="Candlestick")


source = ColumnDataSource(data=png)
inc = png.Close > png.Open
inc = CDSView(source=source, filters=[BooleanFilter(inc)])
dec = png.Open > png.Close
dec = CDSView(source=source, filters=[BooleanFilter(dec)])
w = 24*60*60*1000*30  # day in ms

hover = HoverTool(
    tooltips=[
        ("date", "@date{%F}"),
        ("open", "@Open"),
        ("high", "@High"),
        ("close", "@Close"),
    ],
    formatters={
        '@date': 'datetime'
    },
)

TOOLS = [CrosshairTool(), hover, BoxZoomTool(), PanTool(), ResetTool()]

p = figure(x_axis_type="datetime", tools=TOOLS, width=800, height=300)
#p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.3

p.segment(source=source, x0="date", y0="High", x1="date", y1="Low", color="black")
p.vbar(source=source, view=inc, width=w, x="date", top='Open',
       bottom='Close', fill_color="green", line_color="black")
p.vbar(source=source, view=dec, width=w, x="date", top='Open',
       bottom='Close', fill_color="red", line_color="black")
p.title = "PNG's Food Pricings from 2009 to 2022"


p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

### Plotly

In [7]:
import plotly.graph_objects as go

candlestick = go.Candlestick(
    x=png.date,
    open=png['Open'],
    high=png['High'],
    low=png['Low'],
    close=png['Close']
)

fig = go.Figure(data=[candlestick])
fig.update_layout(
    width=650, height=400,
    margin=dict(
        l=25, r=25, b=100, t=25, pad=2),
    title="Papua New Guinea",
    yaxis_title='Food Price',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)


fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.layout.font.family = 'Helvetica'

fig.show()
fig.write_html("candlestick-plotly.html")

## By Product & Market

In [92]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fp_mkt = pd.read_csv(
    "https://raw.githubusercontent.com/worldbank/pacific-observatory/main/data/prices/outputs/2009/RTFP_mkt_2009_2022-08-17.csv")

In [82]:
fp_mkt.head(5)

,ISO3,country,adm1_name,adm2_name,mkt_name,lat,lon,geo_id,DATES,year,...,l_imp_tomato,c_imp_tomato,o_imp_watermelons,h_imp_watermelons,l_imp_watermelons,c_imp_watermelons,o_food_price_index,h_food_price_index,l_food_price_index,c_food_price_index
0,PNG,Papua New Guinea,Banz,Banz,Banz,-5.800717,144.623079,gid_-580071446231,01/01/2009,2009,...,4.58,5.61,1.61,1.78,1.46,1.62,0.76,0.82,0.70,0.76
1,PNG,Papua New Guinea,Banz,Banz,Banz,-5.800717,144.623079,gid_-580071446231,01/02/2009,2009,...,4.67,6.18,1.64,1.80,1.48,1.60,0.77,0.82,0.71,0.78
2,PNG,Papua New Guinea,Banz,Banz,Banz,-5.800717,144.623079,gid_-580071446231,01/03/2009,2009,...,5.04,6.43,1.62,1.79,1.45,1.60,0.78,0.85,0.72,0.79
3,PNG,Papua New Guinea,Banz,Banz,Banz,-5.800717,144.623079,gid_-580071446231,01/04/2009,2009,...,5.12,6.44,1.63,1.81,1.45,1.60,0.79,0.86,0.72,0.79
4,PNG,Papua New Guinea,Banz,Banz,Banz,-5.800717,144.623079,gid_-580071446231,01/05/2009,2009,...,5.08,6.20,1.63,1.82,1.44,1.60,0.80,0.87,0.73,0.80


In [83]:
fp_mkt.columns[:40]

Index(['ISO3', 'country', 'adm1_name', 'adm2_name', 'mkt_name', 'lat', 'lon',
       'geo_id', 'DATES', 'year', 'month', 'currency', 'components',
       'start_dense_data', 'last_survey_point', 'data_coverage',
       'data_coverage_recent', 'index_confidence_submodels',
       'spatially_interpolated', 'aibika', 'amaranthus_aupa', 'banana_cooking',
       'banana_ripe', 'broccoli', 'cabbage_pakchoi', 'carrot', 'cassava',
       'choko_tips', 'garlic', 'lettuce', 'orange', 'peanut', 'pineapple',
       'potato', 'pumpkin_tips', 'taro_true', 'tomato', 'watermelons',
       'o_imp_aibika', 'h_imp_aibika'],
      dtype='object')

In [84]:
products = fp_mkt.columns[19:38].tolist()

df = pd.DataFrame()
product_nested = list()
for product in products:
    high_p = str("h_imp_") + str(product)
    low_p = str("l_imp_") + str(product)
    open_p = str("o_imp_") + str(product)
    close_p = str("c_imp_") + str(product)
    product_lst = ["DATES", "mkt_name",
                   product, open_p, high_p, low_p, close_p]
    new_df = fp_mkt[product_lst]
    new_df["product"] = product
    new_df.columns = ["DATES", "mkt_name", "price",
                      "open", "high", "low", "close", "product"]
    df = pd.concat([df, new_df], axis=0)

df["DATES"] = pd.to_datetime(df["DATES"], dayfirst=True)

df["product"] = df["product"].astype("category")
df = df.reset_index().drop("index",axis=1)

In [85]:
df["incdec"] = ["Increase" if df["close"][i] >= df['open'][i] else "Decrease" for i in df.index]
df['middle'] = (df.close + df.open)/2 
df['meight'] = abs(df.close - df.open)

In [46]:
jscode_product = """

    var data = source.data;
    var f = selector_product.value;
    var product = data['product'];
    var data_s = {DATES:[], product:[],
                  open:[], close:[],
                  high:[], low:[]};
                  mkt:[]};

    for (let i = 0; i <= data.index.length; i++){
        if (product[i] == f){
            data_s['open'].push(data['open'][i]);
            data_s['close'].push(data['close'][i]);
            data_s['high'].push(data['high'][i]);
            data_s['low'].push(data['low'][i]);
            data_s['DATES'].push(data['DATES'][i]);
            data_s['product'].push(data['product'][i]);
            data_s['mkt'].push(data['mkt_name'][i]);
            }
    };

    // set the new data on source, BokehJS will pick this up automatically
    source_product.data = data_s; 
    // source_up.change.emit();

"""

In [90]:
jscode_product = """

    var data = source.data;
    var f = selector_product.value;
    var product = data['product'];
    var data_s = {DATES:[], product:[],
                  open:[], close:[],
                  high:[], low:[]}

    for (let i = 0; i <= data.index.length; i++){
        if (product[i] == f){
            data_s['open'].push(data['open'][i]);
            data_s['close'].push(data['close'][i]);
            data_s['high'].push(data['high'][i]);
            data_s['low'].push(data['low'][i]);
            data_s['DATES'].push(data['DATES'][i]);
            data_s['product'].push(data['product'][i]);
            }
    };

    // set the new data on source, BokehJS will pick this up automatically
    source_product.data = data_s; 
    // source_up.change.emit();

"""

In [8]:
jscode_market = """

    var data = source_product.data;
    var f = selector_market.value;
    var mkt = data['mkt'];
    var data_s = {DATES:[],mkt:[],
                  open:[], close:[],
                  high:[], low:[]}

    for (let i = 0; i <= data.index.length; i++){
        if (mkt[i] == f){
            data_s['open'].push(data['open'][i]);
            data_s['close'].push(data['close'][i]);
            data_s['high'].push(data['high'][i]);
            data_s['low'].push(data['low'][i]);
            data_s['DATES'].push(data['DATES'][i]);
            }
    };

    // set the new data on source, BokehJS will pick this up automatically
    source_market.data = data_s; 
    //source_up.change.emit();

"""

selector_market = Select(title="Market", value="all", options=mkt_lst)
selector_market.js_on_change('value', CustomJS(args=dict(source_product=source_product,
                                                         selector_market=selector_market,
                                                         source_market=source_market),
                                               code=jscode_market))


In [91]:
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, \
    Slider, TextInput, HoverTool, CDSView, BooleanFilter, CustomJS
from bokeh.plotting import figure, show, output_file

output_file("candlestick-select.html", title="PNG's Product Pricings")

source = ColumnDataSource(data=df)
source_product = ColumnDataSource(
    dict(DATES=[], product=[], open=[], high=[], low=[], close=[]))


hover = HoverTool(
    tooltips=[
        ('date', '@DATES{%F}'),
        ('open', '@open{0}'),
        ('high', '@high{0}'),
        ('low', '@low{0}'),
        ('close', '@close{0}'),
        ("product", '@product')],

    formatters={
        '@DATES': 'datetime'
    },
    mode='mouse'
)


#inc_b = source_product.data['close'] > source_product.data['open']
#inc = CDSView(source=source_product, filters=[BooleanFilter(inc_b)])
#dec_b = source_product.data['open'] > source_product.data['close']
#dec = CDSView(source=source_product, filters=[BooleanFilter(dec_b)])
w = 60000



col_data_src1 = ColumnDataSource(df[df.incdec=='Increase'])
col_data_src2 = ColumnDataSource(df[df.incdec=='Decrease'])

p = figure(x_axis_type="datetime", height=400, width=800)
p.segment(source=source_product, x0='DATES', x1='DATES',
          y0='high', y1='low', color="black")

p.vbar(source=source_product, x='DATES', width=w, top='open',
       bottom='close', fill_color="green", line_color="black")
p.vbar(source=source_product, x='DATES', width=w, top='open',
       bottom='close', fill_color="red", line_color="black")
p.add_tools(hover)

p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.title = "PNG's Product Pricings Over Time"

# Define your select options and default value
product_lst = df["product"].unique().tolist()
mkt_lst = df["mkt_name"].unique().tolist()
selector_product = Select(title="Proudct", value="all", options=product_lst)
selector_product.js_on_change('value', CustomJS(args=dict(source=source,
                                                          selector_product=selector_product,
                                                          source_product=source_product),
                                                code=jscode_product))



l = column(row(column(selector_product), p))

show(l)